In [1]:
import scanpy as sc

sc.settings.set_figure_params(figsize=(4, 4))
import pandas as pd
import numpy as np
import itertools

In [2]:
adata_all = sc.read_h5ad("../../data/30_merge_adata/adata_scvi.h5ad")
adata_malignant_b = sc.read_h5ad("../../data/60_cluster_analysis/adata_malignant_b_cells.h5ad")

In [3]:
adata = adata_all[adata_all.obs["cell_type"] == "malignant B cell", :].copy()

In [4]:
adata.obs

,patient,timepoint,sample,n_genes_by_counts,total_counts,total_counts_mito,pct_counts_mito,n_counts,n_genes,patient_id,sex,age,ethnicity,response,leiden,leiden_scvi,cell_type
cell_id,,,,,,,,,,,,,,,,,
646419_0,P1,T1,HCL_P1_T1,2757,11779.0,2893.0,24.560659,11779.0,2757,P1,female,85,caucasian,short_term,10,0,malignant B cell
825469_0,P1,T1,HCL_P1_T1,2607,10887.0,3520.0,32.332142,10887.0,2607,P1,female,85,caucasian,short_term,10,1,malignant B cell
818696_0,P1,T1,HCL_P1_T1,2604,10103.0,3070.0,30.387014,10103.0,2604,P1,female,85,caucasian,short_term,10,4,malignant B cell
41552_0,P1,T1,HCL_P1_T1,2383,9685.0,2996.0,30.934435,9685.0,2383,P1,female,85,caucasian,short_term,10,4,malignant B cell
467785_0,P1,T1,HCL_P1_T1,2561,10090.0,2453.0,24.311199,10090.0,2561,P1,female,85,caucasian,short_term,10,0,malignant B cell
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
820086_11,P5,T0,HCL_P5_T0,793,2050.0,777.0,37.902439,2050.0,793,P5,male,56,caucasian,long_term,2,2,malignant B cell
133321_11,P5,T0,HCL_P5_T0,780,2036.0,800.0,39.292732,2036.0,780,P5,male,56,caucasian,long_term,2,5,malignant B cell
222098_11,P5,T0,HCL_P5_T0,750,2130.0,723.0,33.943661,2130.0,750,P5,male,56,caucasian,long_term,2,2,malignant B cell


In [5]:
adata.obs.groupby(
    ["patient", "timepoint", "sample", "response"], observed=True
).size().reset_index(name="size").sort_values(["timepoint", "patient"])

,patient,timepoint,sample,response,size
7,P2,T0,HCL_P2_T0,short_term,3350
4,P3,T0,HCL_P3_T0,short_term,1093
8,P4,T0,HCL_P4_T0,long_term,3782
10,P5,T0,HCL_P5_T0,long_term,5211
9,P6,T0,HCL_P6_T0,long_term,4186
0,P1,T1,HCL_P1_T1,short_term,383
5,P2,T1,HCL_P2_T1,short_term,2329
2,P3,T1,HCL_P3_T1,short_term,6352
1,P1,T2,HCL_P1_T2,short_term,37
6,P2,T2,HCL_P2_T2,short_term,2374


## Comparison between short- and long-term responders
* Only T0
* All timepoints

In [6]:
def make_samplesheet(adata):
    """Make a sample sheet, compatible with the DESeq2 ICBI script"""
    deseq_sample_sheet = (
        adata.obs.loc[:, ["patient", "response", "sex", "age"]]
        .sort_values("patient")
        .drop_duplicates()
    )
    return deseq_sample_sheet


def make_bulk(adata, filename_base):
    """Make a bulk data frame, compatible with the DESeq2 ICBI script"""
    bulk_samples = {}
    patients = adata.obs["patient"].unique()
    for sample in patients:
        bulk_samples[sample] = pd.Series(
            np.sum(
                adata[adata.obs["patient"] == sample, :].layers["raw_counts"],
                axis=0,
            ).A1,
            index=adata.var_names,
        )

    bulk_df = pd.DataFrame(bulk_samples)
    bulk_df.index.name = "gene_id"
    bulk_df.reset_index(inplace=True)
    bulk_df.insert(1, "gene_name", bulk_df["gene_id"])
    bulk_df.to_csv(f"{filename_base}_bulk_df.tsv", sep="\t", index=False)
    samplesheet = make_samplesheet(adata)
    samplesheet.to_csv(f"{filename_base}_samplesheet.csv", index=False)

In [7]:
bulk_all_timepoints = make_bulk(
    adata, "../../data/70_de_analysis/71_make_pseudobulk/bulk_response_all_timepoints"
)
bulk_t0 = make_bulk(
    adata[adata.obs["timepoint"] == "T0", :],
    "../../data/70_de_analysis/71_make_pseudobulk/bulk_response_t0",
)

## Each short-term responder individually vs all long-term responders (single-cell level)

In [8]:
def prepare_for_de_short_vs_long_term(adata, file_id):
    short_term_responder = np.unique(
        adata.obs.loc[adata.obs["response"] == "short_term", "patient"].values
    )
    long_term_responder = np.unique(
        adata.obs.loc[adata.obs["response"] == "long_term", "patient"].values
    )

    for tmp_patient in short_term_responder:
        tmp_adata = adata[
            adata.obs["patient"].isin(list(long_term_responder) + [tmp_patient]), :
        ]
        expr = pd.DataFrame(
            tmp_adata.layers["raw_counts"].toarray().T,
            index=tmp_adata.var_names,
            columns=tmp_adata.obs_names,
        )
        expr.reset_index(inplace=True)
        expr.insert(1, "gene_name", expr["gene_id"])
        expr.to_csv(
            f"../../data/70_de_analysis/71_make_pseudobulk/sc_response_{file_id}_{tmp_patient}_vs_long_term.tsv",
            sep="\t",
            index=False,
        )
        tmp_adata.obs.reset_index().to_csv(
            f"../../data/70_de_analysis/71_make_pseudobulk/sc_response_{file_id}_{tmp_patient}_vs_long_term_samplesheet.csv",
            index=False,
        )

In [ ]:
prepare_for_de_short_vs_long_term(adata, "all_timepoints")
prepare_for_de_short_vs_long_term(adata[adata.obs["timepoint"] == "T0", :], "T0")

## Healthy vs. malignant B cell (single-cell level, all timepoints)

In [ ]:
tmp_adata = adata_all[
    adata_all.obs["cell_type"].isin(["healthy B cell", "malignant B cell"])
]
tmp_adata.obs["cell_type"] = [
    "healthy" if ct == "healthy B cell" else "malignant"
    for ct in tmp_adata.obs["cell_type"]
]
expr = pd.DataFrame(
    tmp_adata.layers["raw_counts"].toarray().T,
    index=tmp_adata.var_names,
    columns=tmp_adata.obs_names,
)
expr.index.name = "gene_id"
expr.reset_index(inplace=True)
expr.insert(1, "gene_name", expr["gene_id"])
expr.to_csv(
    f"../../data/70_de_analysis/71_make_pseudobulk/sc_healthy_vs_malignant_b_cells.tsv",
    sep="\t",
    index=False,
)
tmp_adata.obs.reset_index().to_csv(
    f"../../data/70_de_analysis/71_make_pseudobulk/sc_healthy_vs_malignant_b_cells_samplesheet.csv",
    index=False,
)

## DE analysis of timepoints (pseudobulk, only P2-3)

In [ ]:
def make_bulk_timepoints(adata, filename_base):
    """Make a bulk data frame, compatible with the DESeq2 ICBI script"""
    bulk_samples = {}
    patients = adata.obs["patient"].unique()
    timepoints = adata.obs["timepoint"].unique()
    samplesheet = []
    for patient, timepoint in itertools.product(patients, timepoints):
        bulk_samples[f"{patient}_{timepoint}"] = pd.Series(
            np.sum(
                adata[
                    (adata.obs["patient"] == patient)
                    & (adata.obs["timepoint"] == timepoint),
                    :,
                ].layers["raw_counts"],
                axis=0,
            ).A1,
            index=adata.var_names,
        )
        samplesheet.append(
            {
                "sample": f"{patient}_{timepoint}",
                "patient": patient,
                "timepoint": "pre-treatment" if timepoint == "T0" else "post-treatment",
            }
        )

    bulk_df = pd.DataFrame(bulk_samples)
    bulk_df.index.name = "gene_id"
    bulk_df.reset_index(inplace=True)
    bulk_df.insert(1, "gene_name", bulk_df["gene_id"])
    bulk_df.to_csv(f"{filename_base}_bulk_df.tsv", sep="\t", index=False)
    samplesheet = pd.DataFrame.from_records(samplesheet)
    samplesheet.to_csv(f"{filename_base}_samplesheet.csv", index=False)

In [ ]:
bulk_timepoints = make_bulk_timepoints(
    adata[adata.obs["patient"].isin(["P2", "P3"]), :],
    "../../data/70_de_analysis/71_make_pseudobulk/bulk_timepoints",
)

## Comparison of JUN+ cluster vs rest of malignant B cells

In [ ]:
tmp_adata = adata_malignant_b
expr = pd.DataFrame(
    tmp_adata.layers["raw_counts"].toarray().T,
    index=tmp_adata.var_names,
    columns=tmp_adata.obs_names,
)
expr.index.name = "gene_id"
expr.reset_index(inplace=True)
expr.insert(1, "gene_name", expr["gene_id"])
expr.to_csv(
    f"../../data/70_de_analysis/71_make_pseudobulk/sc_fosb_b_cells.tsv",
    sep="\t",
    index=False,
)
tmp_adata.obs.reset_index().to_csv(
    f"../../data/70_de_analysis/71_make_pseudobulk/sc_fosb_b_cells_samplesheet.csv",
    index=False,
)